In [21]:
# seq2seq를 이용해 글자 단위 번역기(영어->한국어)
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64
epochs = 100
latent_dim = 1024  # 인코딩 공간의 잠재 차원
data_path = 'kor.txt'
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
lines = open(data_path, encoding='utf8').read().split('\n')
# print(len(lines))

5893


In [25]:
for line in lines[:len(lines) - 1]:
  input_text, target_text, _ = line.split('\t')
  target_text = '\t' + target_text + '\n'  # tab을 문장의 시작<Start Of Sentence>, enter를 문장의 끝<End Of Sentence>
  input_texts.append(input_text)
  target_texts.append(target_text)  # 영어
  for char in input_text:   # 영어
    if char not in input_characters:
      input_characters.add(char)   # 중복을 배제한 영어 글자 기억

  for char in target_text:   # 영어
    if char not in target_characters:
      target_characters.add(char)   # 중복을 배제한 한국어 글자 기억

print('input_texts : ', input_texts)
print('target_texts : ', target_texts)
print('input_characters : ',input_characters)
print('target_characters : ',target_characters)

# 리스트로 바꾸기
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

print('특수문자 포함 영어 관련 글자 수 : ', num_encoder_tokens)
print('특수문자 포함 영어 관련 글자 : ', input_characters)

print('특수문자 포함 한국어 관련 글자 수 : ', num_decoder_tokens)
print('특수문자 포함 한국어 관련 글자 : ', target_characters)

max_encoder_seq_len = max([len(t) for t in input_texts])
max_decoder_seq_len = max([len(t) for t in target_texts])
print('영어 단어 중 가장 긴 단어 글자 수 : ', max_encoder_seq_len)
print('한국어 단어 중 가장 긴 단어 글자 수 : ', max_decoder_seq_len)

# 글자 집합에 글자 단위로 저장
input_token_index = dict([(char, i) for i, char in enumerate(input_characters) ])
print(input_token_index)  # 영어 :  {' ': 0, '!': 1, '"': 2, '$': 3, '%': 4, "'": 5, ',': 6, '-': 7,

target_token_index = dict([(char, i) for i, char in enumerate(target_characters) ])
print(target_token_index) # 한국어 : {'\t': 0, '\n': 1, ' ': 2, '!': 3, '"': 4, '%': 5, '(': 6, ')': 7,

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_len, num_encoder_tokens), dtype='float32')
# print('encoder_input_data : ', encoder_input_data) # [[[0. 0. 0. ... 0. 0. 0.]  [0. 0. 0. ... 0. 0. 0.]
print(encoder_input_data.shape)   # (5892, 537, 74)

decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_len, num_decoder_tokens), dtype='float32')
print(decoder_input_data.shape)   # (5892, 298, 1010)

decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_len, num_decoder_tokens), dtype='float32')
print(decoder_target_data.shape)  # (5892, 298, 1010)

# 0으로 채워진 배열에 해당 글자가 있는 지점에는 1을 기억
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  # print(i, input_text, target_text)  # 9284 Please tidy up your bedroom. 	네 침실 좀 정리해라.
  for t, char in enumerate(input_text): # t:인덱스값, char:characters값
    encoder_input_data[i, t, input_token_index[char]] = 1. # 해당 글자가 있는 경우 1을 기억
  for t, char in enumerate(target_text): # t:인덱스값, char:characters값
    decoder_input_data[i, t, target_token_index[char]] = 1. # 해당 글자가 있는 경우 1을 기억
    if t > 0:
      decoder_target_data[i, t - 1, target_token_index[char]] = 1.  # decoder_target_data가 한 step 앞서 진행됨. 시작 문자는 포함되지 않음
  print(encoder_input_data[[1]])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 

In [ ]:
# 입력 시퀀스를 정의 후 처리 : Functional api 사용
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state = True) # return_sequences = True/False에 상관없이 마지막 은닉상태를 출력함
encoder_outputs, state_h, state_c = encoder(encoder_inputs)  # state_h:은닉 상태, state_c:셀 상태

# encoder_outputs은 버리고 state_h, state_c만 유지 : context vector
encoder_states = [state_h, state_c]  # 이게 바로 context vector(문맥 벡터)!!